In [ ]:
import matplotlib.pyplot as plt

from pynq import Overlay
import numpy as np

from qsystem_2 import *
from qsystem2_asm import *

In [1]:
outputChannel = 5 # choose 1 through 7 here
inputChannel = 0 # choose 0 or 1 here

In [ ]:
with ASM_Program() as p:
    p.memri(1,0,0, "Number of frequencies") 
    p.memri(0,1,1, "Start/current DAC Frequency")
    p.memri(0,2,2, "Delta DAC Frequency")
    p.memri(0,3,3, "Output Gain")
    p.memri(0,4,4, "Pulse Width")
    p.memri(0,5,5, "Delay Width")
    p.memri(0,6,6, "Start/current ADC Frequency")
    p.memri(0,7,7, "Delta ADC Frequency")
    p.memri(0,8,8, "Readout Trigger Offset")
    p.memri(0,9,9, "Readout Trigger Duration")
    p.math(0,9,8,"+",9) #Add the trigger offset to the trigger duration to get the time when we want to turn the trigger off. 
    p.memri(1,1,10 "Number of iterations per frequency")
    
    p.regwi(0,10, 0xC001) #Trigger value
    p.regwi(0,11,0b1001,"0b1001, stdysel = 1 (zero value), mode = 0 (nsamp), outsel = 01 (DDS).") #Output Mode
    p.bitwi(0,11,11, "<<", 16)
    p.bitw(0,11,11,"|",4)
    
    p.synci(1000) #This is actually really nice because the processor will que up all the outputs before actually starting anything. 
    
    p.label("FREQ_LOOP")
    
    p.label("ITTR_LOOP")
    
    p.set(outputChannel,0,1,0,0,3,11,0) #Start the output pulse
    p.set(inputChannel + 6,0,6,0,0,0,0,0) #Set the freuqnecy of the input DDS
    p.set(0,0,10,0,0,0,0,8) #Start the readout pulse after trigger offset
    p.set(0,0,0,0,0,0,0,9) #End the trigger after the trigger druation
    p.sync(0,4) #Wait for the pulse to be sent. 
    p.sync(0,5) #Wait for the inter-pulse delay
    
    p.loopnz(1,1,"ITTR_LOOP")
    
    p.math(0,1,1,"+",2) #Add the delta output frequency to the output frequency register
    p.math(0,6,6,"+",7) #Add the delta adc frequency ot the adc frequency register
    
    p.loopnz(1,0,"FREQ_LOOP")
    
soc.tproc.load_asm_program(p)

In [4]:
def freqSweep(
    freqStart = 1000,
    deltaFreq = 1,
    nFreqs = 1000,
    iterations = 10,
    gain = 32767,
    pulseWidth = 100,
    delayWidth = 50,
    tOffset = 214, 
    tDuration = 5, 
    ):
    
    freqStartDAC = freq2reg(soc.fs_dac, freqStart, B=32)
    freqDeltaDAC = freq2reg(soc.fs_dac, deltaFreq, B=32)
    freqStartADC = None ######################################TODO: Find out how to format this code...
    freqDeltaADC = None ######################################TODO: Find out how to format this code...
    
    soc.tproc.single_write(addr = 0, data = nFreqs)
    soc.tproc.single_write(addr = 1, data = freqStartDAC)
    soc.tproc.single_write(addr = 2, data = freqDeltaDAC)
    soc.tproc.single_write(addr = 3, data = gain)
    soc.tproc.single_write(addr = 4, data = pulseWidth)
    soc.tproc.single_write(addr = 5, data = delayWidth)
    soc.tproc.single_write(addr = 6, data = freqStartADC)
    soc.tproc.single_write(addr = 7, data = freqDeltaADC)
    soc.tproc.single_write(addr = 8, data = tOffset)
    soc.tproc.single_write(addr = 9, data = tDuration)
    soc.tproc.single_write(addr = 10, data = iterations)
    
    bufLen = iterations * nFreqs
    
    soc.readouts[inputChannel].set_out("product")
    soc.readouts[inputChannel].set_freq(cavFreq) #########################TODO: Change this to onfly
    soc.avg_bufs[inputChannel].config(address=inputChannel, length=pulseWidth)
    soc.avg_bufs[inputChannel].enable()
    
    soc.tproc.stop()
    soc.tproc.start()
    
    time.sleep(1)
    
    iacc, qacc = soc.get_accumulated(ch=inputChannel, length=bufLen)
    
    return iacc, qacc